In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/post-processed.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/post-processed/Michael_Goldrich/Michael_Goldrich_0001_0000.jpg  
  inflating: /content/post-processed/Michael_Goldrich/Michael_Goldrich_0001_0001.jpg  
  inflating: /content/post-processed/Michael_Goldrich/Michael_Goldrich_0002_0000.jpg  
  inflating: /content/post-processed/Michael_Goldrich/Michael_Goldrich_0002_0001.jpg  
  inflating: /content/post-processed/Michael_Goldrich/Michael_Goldrich_0002_0002.jpg  
   creating: /content/post-processed/Michael_Hagee/
  inflating: /content/post-processed/Michael_Hagee/Michael_Hagee_0001_0000.jpg  
  inflating: /content/post-processed/Michael_Hagee/Michael_Hagee_0002_0000.jpg  
   creating: /content/post-processed/Michael_Haneke/
  inflating: /content/post-processed/Michael_Haneke/Michael_Haneke_0001_0000.jpg  
  inflating: /content/post-processed/Michael_Haneke/Michael_Haneke_0001_0001.jpg  
  inflating: /content/post-processed/Michael_Haneke/Michael_Haneke_0002_0000.jpg 

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from IPython.display import Image, display

In [ ]:
train_dir = '../content/post-processed'

In [ ]:
img_width = 112
img_height = 112
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 10858 images belonging to 2996 classes.
Found 1142 images belonging to 2996 classes.


In [ ]:
def create_model(img_width, img_height, num_classes):
    inputs = Input(shape=(img_width, img_height, 3))

    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = create_model(img_width, img_height, num_classes=2996)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=42,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/42


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


340/340 ━━━━━━━━━━━━━━━━━━━━ 568s 2s/step - accuracy: 0.0018 - loss: 8.1104 - val_accuracy: 0.0018 - val_loss: 8.1989 - learning_rate: 0.0010
Epoch 2/42
340/340 ━━━━━━━━━━━━━━━━━━━━ 562s 2s/step - accuracy: 0.0088 - loss: 7.7171 - val_accuracy: 0.0175 - val_loss: 7.5926 - learning_rate: 0.0010
Epoch 3/42
340/340 ━━━━━━━━━━━━━━━━━━━━ 566s 2s/step - accuracy: 0.0155 - loss: 7.3335 - val_accuracy: 0.0210 - val_loss: 7.2950 - learning_rate: 0.0010
Epoch 4/42
340/340 ━━━━━━━━━━━━━━━━━━━━ 565s 2s/step - accuracy: 0.0218 - loss: 6.9843 - val_accuracy: 0.0368 - val_loss: 6.6546 - learning_rate: 0.0010
Epoch 5/42
340/340 ━━━━━━━━━━━━━━━━━━━━ 612s 2s/step - accuracy: 0.0359 - loss: 6.5613 - val_accuracy: 0.0368 - val_loss: 6.5669 - learning_rate: 0.0010
Epoch 6/42
340/340 ━━━━━━━━━━━━━━━━━━━━ 545s 2s/step - accuracy: 0.0491 - loss: 6.2216 - val_accuracy: 0.0578 - val_loss: 6.3508 - learning_rate: 0.0010
Epoch 7/42
340/340 ━━━━━━━━━━━━━━━━━━━━ 576s 2s/step - accuracy: 0.0708 - loss: 5.8118 - val_

In [ ]:
model.save("ModelFaceRecognition.h5")